In [ ]:
import os
import glob

import pandas as pd
import numpy as np
from pathlib import Path

import random
from tqdm.notebook import tqdm
import pydicom # Handle MRI images

import cv2  # OpenCV - https://docs.opencv.org/master/d6/d00/tutorial_py_root.html

from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.metrics import roc_auc_score


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers


In [ ]:
import tensorflow.keras.layers as tkl

In [ ]:

from tensorflow import keras

In [ ]:
from pathlib import Path
data_dir = Path('../input/rsna-miccai-brain-tumor-radiogenomic-classification/')
test_df = pd.read_csv(data_dir / "sample_submission.csv")

In [ ]:
mri_types = ["FLAIR", "T1w", "T2w", "T1wCE"]
excluded_images = [109, 123, 709] # Bad images

SEED_LENGTH = 10 # Number of seeds to use for the model ensemble

In [ ]:
model = keras.models.load_model('../input/bestmodel/best_model_121.h5')

In [ ]:
testlist=os.listdir('../input/rsna-miccai-brain-tumor-radiogenomic-classification/test')

In [ ]:
def load_dicom(path, size = 224):
    ''' 
    Reads a DICOM image, standardizes so that the pixel values are between 0 and 1, then rescales to 0 and 255
    
    Not super sure if this kind of scaling is appropriate, but everyone seems to do it. 
    '''
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    # transform data into black and white scale / grayscale
#     data = data - np.min(data)
    if np.max(data) != 0:
        data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return cv2.resize(data, (size, size))
def get_all_image_paths(brats21id, image_type, folder='train'): 
    '''
    Returns an arry of all the images of a particular type for a particular patient ID
    '''
    assert(image_type in mri_types)
    
    patient_path = os.path.join(
        "../input/rsna-miccai-brain-tumor-radiogenomic-classification/%s/" % folder, 
        str(brats21id).zfill(5),
    )

    paths = sorted(
        glob.glob(os.path.join(patient_path, image_type, "*")), 
        key=lambda x: int(x[:-4].split("-")[-1]),
    )
    
    num_images = len(paths)
    
    start = int(num_images * 0.25)
    end = int(num_images * 0.75)

    interval = 3
    
    if num_images < 10: 
        interval = 1
    
    return np.array(paths[start:end:interval])

def get_all_images(brats21id, image_type, folder='train', size=225):
    return [load_dicom(path, size) for path in get_all_image_paths(brats21id, image_type, folder)]

In [ ]:
from tqdm import tqdm
def get_all_data_for_test(image_type, image_size=32):
    global test_df
    
    X = []
    test_ids = []

    for i in tqdm(test_df.index):
        x = test_df.loc[i]
        images = get_all_images(int(x['BraTS21ID']), image_type, 'test', image_size)
        X += images
        test_ids += [int(x['BraTS21ID'])] * len(images)

    return np.array(X), np.array(test_ids)

In [ ]:
X_test, testidt = get_all_data_for_test('FLAIR', image_size=32)

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam')

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
pred = np.argmax(y_pred, axis=1)

In [ ]:
submission=pd.DataFrame()

In [ ]:
submission['BraTS21ID']=testidt
submission['MGMT_value']=pred
finsub=submission.sort_values(by=['BraTS21ID'])
finsub.to_csv('submission.csv',index=False)